In [ ]:
%load_ext autoreload
%autoreload 2

from dataset_prepare import *
from utils import custom_RFC, print_DI
import pickle

from matplotlib import pyplot as plt

from aif360.metrics import BinaryLabelDatasetMetric
from aif360.datasets.standard_dataset import StandardDataset
from aif360.algorithms.preprocessing import Reweighing, DisparateImpactRemover
from aif360.algorithms.preprocessing.lfr import LFR
from aif360.metrics import ClassificationMetric
from sklearn.metrics import classification_report
from aif360.algorithms.preprocessing.optim_preproc import OptimPreproc
from aif360.algorithms.preprocessing.optim_preproc_helpers.opt_tools import OptTools

from IPython.display import Markdown, display

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

from tqdm import tqdm
import imblearn

In [ ]:
df = load_dataset()
df

In [ ]:
label = 'mortal'

# Récupération des ensembles de train/test
X_train, X_test, y_train, y_test = test_train_sets(df)
df = df.drop(columns='Num_Acc')

In [ ]:
dataset_orig_train, dataset_orig_test = prepare_standard_dataset(X_train, y_train, X_test, y_test, label)

In [ ]:
# print out some labels, names, etc.
display(Markdown("#### Training Dataset shape"))
print(dataset_orig_train.features.shape)
display(Markdown("#### Favorable and unfavorable labels"))
print(dataset_orig_train.favorable_label, dataset_orig_train.unfavorable_label)
display(Markdown("#### Protected attribute names"))
print(dataset_orig_train.protected_attribute_names)
display(Markdown("#### Privileged and unprivileged protected attribute values"))
print(dataset_orig_train.privileged_protected_attributes, 
      dataset_orig_train.unprivileged_protected_attributes)
display(Markdown("#### Dataset feature names"))
print(dataset_orig_train.feature_names)

In [ ]:
privileged_groups=[{'sexe_conducteur': 1}]
unprivileged_groups = [{'sexe_conducteur' : 0}]

## Repondération
Utilisation de l'outil de repondération de `aif360` et affichage du résultat obtenu.

In [ ]:
# Metric for the original dataset
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

In [ ]:
# 0 -> Femme et 1 -> Homme
RW = Reweighing(privileged_groups=privileged_groups, unprivileged_groups=unprivileged_groups)
RW.fit(dataset_orig_train)
dataset_transf_train = RW.transform(dataset_orig_train)

In [ ]:
metric_transf_train = BinaryLabelDatasetMetric(dataset_transf_train, 
                                         unprivileged_groups=unprivileged_groups,
                                         privileged_groups=privileged_groups)
display(Markdown("#### Transformed training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_train.mean_difference())

In [ ]:
# valeurs catégorielles
categorical_features = ['trajet', 'catr', 'circ', 'nbv', 'prof',
                        'plan', 'surf', 'vma', 'lum', 'agg', 
                        'int', 'atm', 'col', 'catv', 'obs', 'obsm', 'choc', 'pieton',
                        'sexe_conducteur', 'infra', 'situ']
# valeurs numériques
numerical_features = ['dep','age', 'mois']

print("numerical : ", numerical_features)
print("categorical : ", categorical_features)

In [ ]:
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

transformations = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)])

In [ ]:
X_train = dataset_transf_train.convert_to_dataframe()[0]
y_train = dataset_transf_train.convert_to_dataframe()[0]['mortal']

In [ ]:
from utils import custom_RFC
import imblearn

dt = custom_RFC(random_state=42)
clf = imblearn.pipeline.Pipeline(
    [
        ('preprocessor', transformations),
        ("resample", imblearn.over_sampling.SMOTE(random_state=42)),
        ('classifier', dt)
    ])
clf = clf.fit(X_train, y_train, classifier__sample_weight=dataset_orig_train.instance_weights)

preds = clf.predict(X_test)

clf.score(X_train, y_train), clf.score(X_test, y_test)

In [ ]:
pickle.dump(clf, open('models/rfc_reweight_model.sav', 'wb'))

In [ ]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, preds).ravel()
tn, fp, fn, tp

In [ ]:
import plotly.express as px

fig = px.imshow([[tn, fp], [fn, tp]], text_auto=True, labels=dict(y="Truth", x="Pred"),
                x=["False", "True"],
                y=["False", "True"]
               )
fig.show()

In [ ]:
compute_metrics(dt, )

## Disparate impact remover
Utilisation de l'outil de `Disparate impact remover` de `aif360` et affichage du résultat obtenu.

In [ ]:
label = 'mortal'

df = load_dataset()
# Récupération des ensembles de train/test
X_train, X_test, y_train, y_test = test_train_sets(df)

In [ ]:
dataset_orig_train, dataset_orig_test = prepare_standard_dataset(X_train, y_train, X_test, y_test, label, transform=True)

In [ ]:
protected = 'sexe_conducteur'

index = dataset_orig_train.feature_names.index(protected)

DIs = []
for level in tqdm(np.linspace(0., 1., 11)):
    di = DisparateImpactRemover(repair_level=level)
    train_repd = di.fit_transform(dataset_orig_train)
    test_repd = di.fit_transform(dataset_orig_test)

    X_tr = np.delete(train_repd.features, index, axis=1)
    X_te = np.delete(test_repd.features, index, axis=1)
    
    y_tr = train_repd.labels.ravel()
    
    dt = custom_RFC(random_state=42)
    dt.fit(X_tr, y_tr)
    
    test_repd_pred = test_repd.copy()
    test_repd_pred.labels = dt.predict(X_te)

    p = [{protected: 1}]
    u = [{protected: 0}]
    cm = BinaryLabelDatasetMetric(test_repd_pred, privileged_groups=p, unprivileged_groups=u)
    DIs.append(cm.disparate_impact())

In [ ]:
protected = 'sexe_conducteur'

index = dataset_orig_train.feature_names.index(protected)

di = DisparateImpactRemover(repair_level=0.9)
train_repd = di.fit_transform(dataset_orig_train)
test_repd = di.fit_transform(dataset_orig_test)

X_tr = np.delete(train_repd.features, index, axis=1)
X_te = np.delete(test_repd.features, index, axis=1)

y_tr = train_repd.labels.ravel()
y_te = test_repd.labels.ravel()

dt = custom_RFC(random_state=42)
dt.fit(X_tr, y_tr)

test_repd_pred = test_repd.copy()
preds = dt.predict(X_te)


In [ ]:
dt.score(X_tr, y_tr), dt.score(X_te, y_te)

In [ ]:
X_train = train_repd.convert_to_dataframe()[0]
X_test = test_repd.convert_to_dataframe()[0]
y_train = train_repd.convert_to_dataframe()[0]['mortal']
y_test = test_repd.convert_to_dataframe()[0]['mortal']

In [ ]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, preds).ravel()
tn, fp, fn, tp

In [ ]:
import plotly.express as px

fig = px.imshow([[tn, fp], [fn, tp]], text_auto=True, labels=dict(y="Truth", x="Pred"),
                x=["False", "True"],
                y=["False", "True"]
               )
fig.show()

### Apprentissage de `représentation latente fair`

In [ ]:
TR = LFR(unprivileged_groups=unprivileged_groups,
         privileged_groups=privileged_groups,
         k=10, Ax=0.1, Ay=1.0, Az=2.0,
         verbose=1
        )
TR = TR.fit(dataset_orig_train, maxiter=5000, maxfun=5000)

In [ ]:
# Transform training data and align features
dataset_transf_train = TR.transform(dataset_orig_train)
dataset_transf_test = TR.transform(dataset_orig_test)

In [ ]:
print(classification_report(dataset_orig_test.labels, dataset_transf_test.labels))

In [ ]:
dataset_transf_test.features

In [ ]:
dataset_orig_test.features

### In processing `Adversarial debiasing`

In [ ]:
from aif360.algorithms.inprocessing.adversarial_debiasing import AdversarialDebiasing
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

In [ ]:
# Load post-processing algorithm that equalizes the odds
# Learn parameters with debias set to False
sess = tf.Session()
plain_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='plain_classifier',
                          debias=False,
                          sess=sess)
plain_model.fit(dataset_transf_train)

In [ ]:
# Apply the plain model to test data
dataset_nodebiasing_train = plain_model.predict(dataset_transf_train)
dataset_nodebiasing_test = plain_model.predict(dataset_orig_test)

In [ ]:
# Metrics for the dataset from plain model (without debiasing)
display(Markdown("#### Plain model - without debiasing - dataset metrics"))
metric_dataset_nodebiasing_train = BinaryLabelDatasetMetric(dataset_nodebiasing_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_train.mean_difference())

metric_dataset_nodebiasing_test = BinaryLabelDatasetMetric(dataset_nodebiasing_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_test.mean_difference())

display(Markdown("#### Plain model - without debiasing - classification metrics"))
classified_metric_nodebiasing_test = ClassificationMetric(dataset_orig_test, 
                                                 dataset_nodebiasing_test,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_nodebiasing_test.accuracy())
TPR = classified_metric_nodebiasing_test.true_positive_rate()
TNR = classified_metric_nodebiasing_test.true_negative_rate()
bal_acc_nodebiasing_test = 0.5*(TPR+TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_nodebiasing_test)
print("Test set: Disparate impact = %f" % classified_metric_nodebiasing_test.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_nodebiasing_test.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_nodebiasing_test.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_nodebiasing_test.theil_index())

In [ ]:
sess.close()
tf.reset_default_graph()
sess = tf.Session()

debiased_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='debiased_classifier',
                          debias=True,
                          sess=sess)

In [ ]:
debiased_model.fit(dataset_transf_train)

In [ ]:
# Apply the plain model to test data
dataset_debiasing_train = debiased_model.predict(dataset_transf_train)
dataset_debiasing_test = debiased_model.predict(dataset_orig_test)

In [ ]:
# Metrics for the dataset from plain model (without debiasing)
display(Markdown("#### Plain model - without debiasing - dataset metrics"))
print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_train.mean_difference())
print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_test.mean_difference())

# Metrics for the dataset from model with debiasing
display(Markdown("#### Model - with debiasing - dataset metrics"))
metric_dataset_debiasing_train = BinaryLabelDatasetMetric(dataset_debiasing_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_debiasing_train.mean_difference())

metric_dataset_debiasing_test = BinaryLabelDatasetMetric(dataset_debiasing_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_debiasing_test.mean_difference())



display(Markdown("#### Plain model - without debiasing - classification metrics"))
print("Test set: Classification accuracy = %f" % classified_metric_nodebiasing_test.accuracy())
TPR = classified_metric_nodebiasing_test.true_positive_rate()
TNR = classified_metric_nodebiasing_test.true_negative_rate()
bal_acc_nodebiasing_test = 0.5*(TPR+TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_nodebiasing_test)
print("Test set: Disparate impact = %f" % classified_metric_nodebiasing_test.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_nodebiasing_test.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_nodebiasing_test.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_nodebiasing_test.theil_index())



display(Markdown("#### Model - with debiasing - classification metrics"))
classified_metric_debiasing_test = ClassificationMetric(dataset_orig_test, 
                                                 dataset_debiasing_test,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_debiasing_test.accuracy())
TPR = classified_metric_debiasing_test.true_positive_rate()
TNR = classified_metric_debiasing_test.true_negative_rate()
bal_acc_debiasing_test = 0.5*(TPR+TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_debiasing_test)
print("Test set: Disparate impact = %f" % classified_metric_debiasing_test.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_debiasing_test.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_debiasing_test.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_debiasing_test.theil_index())

#### Plain model - without debiasing - dataset metrics
Train set: Difference in mean outcomes between unprivileged and privileged groups = 0.082386

Test set: Difference in mean outcomes between unprivileged and privileged groups = 0.011212

#### Model - with debiasing - dataset metrics
Train set: Difference in mean outcomes between unprivileged and privileged groups = 0.082957

Test set: Difference in mean outcomes between unprivileged and privileged groups = 0.010330

#### Plain model - without debiasing - classification metrics
Test set: Classification accuracy = 0.822887

Test set: Balanced classification accuracy = 0.629412

Test set: Disparate impact = 1.068230

Test set: Equal opportunity difference = 0.051500

Test set: Average odds difference = 0.033495

Test set: Theil_index = 0.074225

#### Model - with debiasing - classification metrics
Test set: Classification accuracy = 0.811279

Test set: Balanced classification accuracy = 0.632437

Test set: Disparate impact = 1.057924

Test set: Equal opportunity difference = 0.074621

Test set: Average odds difference = 0.044141

Test set: Theil_index = 0.075338
